In [51]:
import random

host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = 耍一耍 | 玩一玩
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？"""

def get_generation_by_gram(gram_str: str, target, stmt_spilt='=', or_split='|'):
    
    rules=dict()    ##将gram_str信息存入字典rules（stmt&expr）
    
    for line in gram_str.split('\n'):
        if not line:    continue
        stmt, expr = line.split(stmt_spilt)
        rules[stmt.strip()] = expr.split(or_split) 
            
    generated = generate(rules, target=target)
    
    return generated

def generate(gram_rule,target):
    
    if target in gram_rule:     ##names
        candidates = gram_rule[target]    ##name
        candidate = random.choice(candidates)
             
        
        return ''.join(generate(gram_rule, target=c.strip()) for c in candidate.split())
    else:
        return target
    
get_generation_by_gram(host, target='host', stmt_spilt='=', or_split='|')

'你好我是771号,请问你要耍一耍喝酒吗？'

In [31]:
import jieba

from collections import Counter


corpus = 'E:\\NLP\\Works\\1st\\article_9k.txt'
article_file = open(corpus, encoding='utf-8').read()

max_length = 10000
sub_file = article_file[:max_length]

def cut(string):
    return list(jieba.cut(string))

Tokens = cut(sub_file)
word_counts = Counter(Tokens)

_2_gram_Tokens = [Tokens[i]+Tokens[i+1] for i in range(len(Tokens)-1)]
_2_gram_Tokens_counts = Counter(_2_gram_Tokens)

def get_gram_words_counter(word, words_counter):
    if word in words_counter:
        return words_counter[word]
    else:
        return words_counter.most_common()[-1][-1]

def two_grammar_model(sentence):
    tokens = cut(sentence)
    possibility = 1
    
    for i in range(len(tokens)-1):
        word = tokens[i]
        next_word = Tokens[i+1]
        _1_gram_counts = get_gram_words_counter(tokens[i], word_counts)
        _2_gram_counts = get_gram_words_counter(tokens[i]+tokens[i+1], _2_gram_Tokens_counts)
        pro = _2_gram_counts/_1_gram_counts
        
        possibility *= pro
        
    return possibility

two_grammar_model('前天早上吃晚饭的时候')

7.734040140230803e-07

In [148]:
import random
import jieba
from collections import Counter

host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = 耍一耍 | 玩一玩
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？"""

article_site = 'E:\\NLP\\Works\\1st\\train_deal.txt'
article_file = open(article_site, encoding='utf-8').read()

article_max_length = 50000
sub_file = article_file[:article_max_length]

Token_words = cut(sub_file)
one_words_counts = Counter(Token_words)

two_grammar_words = [Token_words[i]+Token_words[i+1] for i in range(len(Token_words)-1)]
two_words_counts = Counter(two_grammar_words)


def get_generation_by_grammar(grammar_str: str, target, stmt_spilt='=', or_split='|'):
    
    rule_dict = dict()
    
    for line in grammar_str.split('\n'):
        if not line:   continue
        stmt, expre = line.split(stmt_spilt)
        rule_dict[stmt.strip()] = expre.split(or_split)
                    
    generated_sentance = generate(rule_dict, target=target)
    
    return generated_sentance

def generate(grammer_rule, target):
    
    if target in grammer_rule:
        candidates = grammer_rule[target]
        candidate  = random.choice(candidates)
        sentance = ''.join(generate(grammer_rule, target=c.strip()) for c in candidate.split())
        
        return sentance
    else:
        return target
    
def generate_n(a):
    sen_list = list()
    for i in range(a):
        sen_list.append(get_generation_by_grammar(host, target='host', stmt_spilt='=', or_split='|'))     
    return sen_list
    

def cut(string_str):
    return list(jieba.cut(string_str))

def get_grammar_word_counts(word, grammar_sentance):
    if word in grammar_sentance:
        return grammar_sentance[word]
    else:
        return grammar_sentance.most_common()[-1][-1]
    
def two_grammar_model(given_sentance):
    token_words = cut(given_sentance)
    possibility = 1
    
    for i in range(len(token_words)-1):
        word = token_words[i]
        word_next = token_words[i]+token_words[i+1]
        one_words_counter = get_grammar_word_counts(word, one_words_counts)
        two_words_counter = get_grammar_word_counts(word_next, two_words_counts)
        pro = two_words_counter/one_words_counter
        possibility *= pro
        
    return possibility


def generate_best(n):
    sentance_possibi_list = list()
    sentance_list = generate_n(n)

    for i in range(n):
        possib = two_grammar_model(sentance_list[i])
        sentance_possibi_list.append([sentance_list[i], possib])
        
    sentance_possibi_sort_list = sorted(sentance_possibi_list, key=lambda x: x[1], reverse=True)
    
    #print(sentance_possibi_sort_list)
    
    return sentance_possibi_sort_list[0]


generate_best(10)



4.0723045304960567e-07

In [143]:
##清洗文本，只剩中文
import re
input_file = r'E:\NLP\Works\1st\train.txt'
output_file = r'E:\NLP\Works\1st\train_deal.txt'  #创建新的文件并写入清洗后的文本

input_file_file = open(input_file, encoding='utf-8').read()
output_str_list = re.findall(r'[\u4e00-\u9fa5]',input_file_file)   #只保留中文
output_str = ''.join(output_str_list)   #字符串形式

output_file_str = open(output_file, 'w', encoding='utf-8')  ##打开清洗后需要写入的文本
output_file_str.write(output_str)

output_file_str.close
    

<function TextIOWrapper.close()>